In [1]:
#import standard libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from random import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv1D, LSTM, Flatten

In [2]:
#import project libs
from PowerAIDataHandler.PowerAIDataHandler import ClassPowerAIDataHandler

#corrupt timestamps: Time frame: 1728835915654/1728845708532/
#  Sprung zwischen 1728835995276 und 1728845640410

In [3]:
#init datahandler using configuration from .env
dh = ClassPowerAIDataHandler("C:/Users/Kids/iCloudDrive/Documents/GitHub/PowerAI/.env")

#init parameters for neural network
num_devices   = len(dh.device_list)   #länge der Gerätliste
event_ratio   = 0.5 #Verhältnis Trainings-/Testdaten
window_length = 15 #Anzahl an Daten die gleichzeitig eingespeist werden
epochs        = 100 #Anzahl der Traings-/Testdurchläufe

In [4]:
#read the calssified events from the mysql database
dh.read_events_from_db()

#print timeserie of the events
#dh.print_events()

In [ ]:
#dh.filter_events_by_minpow()
#dh.print_events()

In [ ]:
x = pd.to_datetime(dh.event_list[32][0]['timestamp'], utc=True, unit='ms')
y = np.array(dh.event_list[32][0]['value']).astype(float)


x = x[0:240]
y = y[0:240]

f = plt.figure()
a = f.add_subplot()
a.plot(x, y, marker = 'o')

In [ ]:
#test_x = { 'timestamp' : np.array([]), 'value' : np.array([]), 'device' : np.array([]) }
#for key in dh.device_list.keys():
 #   for i in range(len(dh.event_list[key])):
  #      test_x['timestamp'] = np.append(test_x['timestamp'], dh.event_list[key][i]['timestamp'])
   #     test_x['value'] = np.append(test_x['value'], dh.event_list[key][i]['value'])
    #    test_x['device'] = np.append(test_x['device'], dh.event_list[key][i]['device'])

In [5]:
test_xx = { 'timestamp' : np.array([]), 'value' : np.array([]), 'device' : np.array([]) }
for key in dh.device_list.keys():
    for i in range(len(dh.event_list[key])):
        test_xx['timestamp'] = np.append(test_xx['timestamp'], dh.event_list[key][i]['timestamp'])
        test_xx['value'] = np.append(test_xx['value'], dh.event_list[key][i]['value'])
        test_xx['device'] = np.append(test_xx['device'], dh.event_list[key][i]['device'])
print(len(test_xx['value']))

167932


In [22]:
test_x = { 'timestamp' : np.array([]), 'value' : np.array([]), 'device' : np.array([]) }
valid_devices = {32}

for i in range(len(test_xx['value'])):
    if test_xx['device'][i] in valid_devices:
        test_x['timestamp'] = np.append(test_x['timestamp'], test_xx['timestamp'][i])
        test_x['value'] = np.append(test_x['value'], test_xx['value'][i])
        test_x['device'] = np.append(test_x['device'], test_xx['device'][i])
print(len(test_x['value']))

72111


In [7]:
#Programm zur Erkennung von kitchen-light
y = np.zeros(len(test_x['value']))
kitchen = 128
start_kitchen = False
for i in range(len(test_x['value']) - 3):  
    if test_x['value'][i+3] - test_x['value'][i] > 280 and test_x['value'][i+3] - test_x['value'][i] < 310:
        y[i+3] = kitchen
        start_kitchen = True
    if test_x['value'][i+3] - test_x['value'][i] > -280 and test_x['value'][i+3] - test_x['value'][i] < -310:
        y[i] = kitchen
        start_kitchen = False
    if start_kitchen == True:
        y[i] = kitchen
        
(y == 128).sum()

139868

In [8]:
#Programm zur Erkennung von oven
#y = np.zeros(len(test_x['value']))
oven = 32
start_oven = False
for i in range(len(test_x['value']) - 150):
    if np.var(test_x['value'][i:i+150]) > 427160 and np.var(test_x['value'][i:i+150]) < 1322000:
        y[i:i+150] = oven
        start_oven = True
    if np.var(test_x['value'][i:i+150]) < 427160 or np.var(test_x['value'][i:i+150]) > 1322000:
        start_oven = False
    if start_oven == True:
        y[i] = oven
(y == 32).sum()

23725

In [9]:
#Programm zur Erkennung von induction-cooker
#y = np.zeros(len(test_x['value']))
induction = 8
start_induction = False
for i in range(len(test_x['value']) - 180):
    if np.var(test_x['value'][i:i+180]) > 361980 and np.var(test_x['value'][i:i+180]) < 1836900:
        y[i:i+180] = induction
        start_induction = True
    if np.var(test_x['value'][i:i+180]) < 361980 or np.var(test_x['value'][i:i+180]) > 1836900:
        start_induction = False
    if start_induction == True:
        y[i] = induction
(y == 8).sum()

27113

In [10]:
#Programm zur Erkennung von espresso-machine
#y = np.zeros(len(test_x['value']))
espresso = 1
start_espresso = False
for i in range(len(test_x['value']) - 3):
    if test_x['value'][i+3] - test_x['value'][i] > 950 and test_x['value'][i+3] - test_x['value'][i] < 1050:
        y[i+3]= espresso
        start_espresso = True
    if test_x['value'][i+3]- test_x['value'][i] > -950 and test_x['value'][i+3] - test_x['value'][i] < -1050:
        y[i] = espresso
        start_espresso = False
    if start_espresso == True:
        y[i] = espresso
        
(y == 1).sum()

141776

In [11]:
#Programm zur Erkennung von microwave
#y = np.zeros(len(test_x['value']))
microwave = 64
start_microwave = False
for i in range(len(test_x['value']) - 3):
    if test_x['value'][i+3] - test_x['value'][i] > 1150 and test_x['value'][i+3] - test_x['value'][i] < 1250:
        y[i+3]= microwave
        start_microwave = True
    if test_x['value'][i+3]- test_x['value'][i] > -1150 and test_x['value'][i+3] - test_x['value'][i] < -1250:
        y[i] = microwave
        start_microwave = False
    if start_microwave == True:
        y[i] = microwave
        
(y == 64).sum()

124591

In [12]:
#Programm zur Erkennung von washing-machine
#y = np.zeros(len(test_x['value']))
washing = 2
start_washing = False
for i in range(len(test_x['value']) - 150):
    if test_x['value'][i+150] - test_x['value'][i] > 2150 and test_x['value'][i+150] - test_x['value'][i] < 2300:
        y[i+150] = washing
        start_washing = True
        start_i = i
    if start_washing == True and i - start_i >= 7200:
        start_washing = False
    if start_washing == True:
        y[i] = washing
(y == 2).sum()

102411

In [13]:
#Programm zur Erkennung von dish-washer
#y = np.zeros(len(test_x['value']))

dish = 4
start_dish = False
for i in range(len(test_x['value'])-6):
    if test_x['value'][i+6] - test_x['value'][i] > 2200 and test_x['value'][i+6] - test_x['value'][i] < 2300:
        y[i+6] = dish
        start_dish = True
        start_i = i
    if start_dish == True and i - start_i >= 6300:
        start_dish = False
    if start_dish == True:
        y[i] = dish
        
(y == 4).sum()

30037

In [ ]:
cnt_wrong = 0
total = len(test_x['value'])
correct = np.zeros(6)
wrong = np.zeros(6)
for i in range(total):
    if test_x['device'][i] ==  and y[i] /= 1:
         = correct[] + 1   
        
        
result_table = PrettyTable(['device name', 'total', 'correct', 'wrong', 'percent'], align='r')        
for i in range(len(dh.device_list)) :
    result_table.add_row([
    dh.device_list[np.power(2, i)]['name'], 
                int(cnt_wrong[i] + cnt_correct[i]),
                int(cnt_correct[i]),
                int(cnt_wrong[i]),
                str(round(100 * cnt_correct[i] / (cnt_correct[i] + cnt_wrong[i]))) + "%" if cnt_correct[i] + cnt_wrong[i] != 0 else ""])

        print(result_table)


In [14]:
#Ergebnis vergleichen

correct = 0
total = len(test_x['value'])
for i in range(total):
    if test_x['device'][i] == y[i]:
        correct = correct + 1

print(f"Die Erfolgsquote beträgt {correct/total*100}%, insgesamt wurden {correct} Messpunkte von {total} korrekt zugeordnet.")

Die Erfolgsquote beträgt 33.17971051259284%, insgesamt wurden 47084 Messpunkte von 141906 korrekt zugeordnet.


In [21]:
#Ergebnis richtig vergleichen
correct = 0
total = len(test_x['value'])
for i in range(total):
    if test_x['device'][i] == 128 and y[i] == 128:
        correct = correct + 1

print(f"Die Erfolgsquote beträgt {correct/total*100}%, insgesamt wurden {correct} Messpunkte von {total} korrekt zugeordnet.")

Die Erfolgsquote beträgt 0.0%, insgesamt wurden 0 Messpunkte von 141906 korrekt zugeordnet.


In [ ]:
for i in range(len(dh.event_list[8])):
    print(f"Varianz von Ofen-Event= {np.var(dh.event_list[8][i]['value'])}")        

In [ ]:
model = Sequential()
# input layer
model.add(Input(shape=(window_length)))
model.add(tf.keras.layers.Reshape(target_shape=(window_length, 1)))
# conv1 layer
model.add(tf.keras.layers.Convolution1D(filters=64, kernel_size=3, padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ReLU())
# conv2 layer
model.add(tf.keras.layers.Convolution1D(filters=64, kernel_size=3, padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ReLU())
# conv3 layer
model.add(tf.keras.layers.Convolution1D(filters=64, kernel_size=3, padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ReLU())
# gap
model.add(tf.keras.layers.GlobalAveragePooling1D())
# output layer
model.add(tf.keras.layers.Dense(num_devices, activation="softmax"))

In [ ]:
import tensorflow as tf
from tensorflow import keras

num_classes = 7

def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=train_x.shape[1:])
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
#xx, yy = dh.generate_training_data_from_db(window_length)
train_x, train_y, test_x, test_y = dh.generate_training_data_from_events(window_length, event_ratio) #fk: aufteilen der Daten

In [ ]:
import keras
input_layer = keras.layers.Input(shape=(train_x.shape[1],1))

conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
conv1 = keras.layers.BatchNormalization()(conv1)
conv1 = keras.layers.ReLU()(conv1)

conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
conv2 = keras.layers.BatchNormalization()(conv2)
conv2 = keras.layers.ReLU()(conv2)

conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
conv3 = keras.layers.BatchNormalization()(conv3)
conv3 = keras.layers.ReLU()(conv3)

gap = keras.layers.GlobalAveragePooling1D()(conv3)

output_layer = keras.layers.Dense(train_y.shape[1], activation="softmax")(gap)

model = keras.models.Model(inputs=input_layer, outputs=output_layer)


In [ ]:
# Kompilieren des Modells
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'msle', 'mae'])
# Anzeigen der Modellzusammenfassung
model.summary()

In [ ]:
# train the model
history = model.fit(
            x=train_x,
            y=train_y,
            batch_size=train_x[0].size,
            epochs=epochs,
            verbose=0)

In [ ]:
plt.plot(history.history['loss'])


In [ ]:
#Prediction with neural network

predict_y = model.predict(test_x)
    
dh.compare_with_testdata(predict_y, test_x, test_y)

In [ ]:
# fixed potential bug in line 218. train data ratio: 0.5, with event filter
#+--------------------+-------+---------+-------+---------+
#|        device name | total | correct | wrong | percent |
#+--------------------+-------+---------+-------+---------+
#|   espresso-machine |   657 |     618 |    39 |     94% |
#|    washing-machine |   474 |       0 |   474 |      0% |
#|        dish-washer |   383 |     382 |     1 |     99% |
#|   induction-cooker |    73 |       0 |    73 |      0% |
#|  irrigation-system |     0 |       0 |     0 |         |
#|               oven |    18 |       7 |    11 |     38% |
#|          microwave |     0 |       0 |     0 |         |
#|      kitchen-light |    21 |       0 |    21 |      0% |
#|  living-room-light |     0 |       0 |     0 |         |
#|  dining-room-light |     0 |       0 |     0 |         |
#| ground-floor-light |    10 |       0 |    10 |      0% |
#|  upper-floor-light |     0 |       0 |     0 |         |
#+--------------------+-------+---------+-------+---------+